In [1]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("ogdf/energybased/FMMMLayout.h")


G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.gridGraph(G, 3, 3, True,True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = "N%s" % n.index()
    GA.x[n] = (n.index() %3) * 50 
    GA.y[n] = (n.index()//3) * 50
    
#or adj in G.nodes[0].adjEntries:
#   print(adj.twinNode().index())
#   if adj.twinNode().index() == 9:
#       e = adj.theEdge()
#       GA.bends[e].pushBack(ogdf.DPoint(-100,-100)) 
    
#SL = ogdf.SugiyamaLayout()
#SL=ogdf.FMMMLayout();
 
#SL.useHighLevelOptions(True);
#SL.unitEdgeLength(15.0);
#SL.newInitialPlacement(True);
#SL.qualityVersusSpeed(ogdf.FMMMOptions.QualityVsSpeed.GorgeousAndEfficient);
#SL.call(GA)
ogdf.GraphIO.write(GA, "sugiyama-simple.graphml")

GA

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [2]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/deterministic.h")

# 0 -- > x
# |
# v
#  y

def add_bends(line, *points):
    for point in points:
        line.pushBack(ogdf.DPoint(*point))

def find_wrap(GA, e):
    s,t = e.source(), e.target()
    wrap_h = GA.x[t] < GA.x[s]
    wrap_v = GA.y[t] < GA.y[s]
    if wrap_h and wrap_v:
        delta_x = abs(GA.x[t] - GA.x[s])
        delta_y = abs(GA.y[t] - GA.y[s])
        if delta_x > delta_y:
            wrap = "hv"
        else:
            wrap = "vh"
    elif wrap_h:
        wrap = "h"
    elif wrap_v:
        wrap = "v"
    else:
        wrap = ""
    return wrap

def apply_wrap(GA, box, e, wrap, margin=50):
    if not wrap:
        return
    
    s,t = e.source(), e.target()
    left = box.p1().m_x
    top = box.p1().m_y
    right = box.p2().m_x
    bottom = box.p2().m_y
    
    GA.bends[e].clear()
    if wrap[0] == "h":
        add_bends(GA.bends[e],
            (right + margin, GA.y[s]),
            (right + margin, top - margin),
            (left - margin, top - margin),
            (left - margin, GA.y[t]),
        )
    elif wrap[0] == "v":
        add_bends(GA.bends[e],
            (GA.x[s], bottom + margin),
            (left - margin, bottom + margin),
            (left - margin, top - margin),
            (GA.x[t], top - margin),
        )


def layout_edges(GA, box):
    wraps = ogdf.EdgeArray[str](GA.constGraph(), "")
    wraps_h = []
    wraps_v = []
    for e in GA.constGraph().edges:
        w = wraps[e] = find_wrap(GA, e)
        if w and w[0] == "h":
            wraps_h.append(e)
        elif w and w[0] == "v":
            wraps_v.append(e)
    wraps_h.sort(key=lambda e: GA.y[e.source()])
    wraps_v.sort(key=lambda e: GA.x[e.source()])
    
    for nr, e in enumerate(wraps_h + wraps_v):
        # GA.arrowType[e] = getattr(ogdf.EdgeArrow, "None")
        apply_wrap(GA, box, e, wraps[e], 50 + nr * 5)
        if wraps[e] in ["hv", "vh"]:
            GA.strokeColor[e] = ogdf.Color(0,255,0)
        elif wraps[e] == "h":
            GA.strokeColor[e] = ogdf.Color(255,0,0)
        elif wraps[e] == "v":
            GA.strokeColor[e] = ogdf.Color(0,0,255)
        
    return GA, wraps

def draw_bounding_box(G, GA, box):
    box_node1 = G.newNode()
    box_node2 = G.newNode()
    GA.x[box_node1] = box.p1().m_x - 25
    GA.y[box_node1] = box.p1().m_y - 25
    GA.x[box_node2] = box.p2().m_x + 25
    GA.y[box_node2] = box.p2().m_y + 25
    GA.width[box_node1] = 0
    GA.height[box_node1] = 0
    GA.width[box_node2] = 0
    GA.height[box_node2] = 0
    box_edge1 = G.newEdge(box_node1, box_node2)
    box_edge2 = G.newEdge(box_node1, box_node2)
    add_bends(GA.bends[box_edge1], (box.p1().m_x - 25, box.p2().m_y + 25))
    add_bends(GA.bends[box_edge2], (box.p2().m_x + 25, box.p1().m_y - 25))

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [3]:
width = 4
height = 4

G = ogdf.Graph()
ogdf.gridGraph(G, width, height, True, True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = str(n.index())
    GA.x[n] = (n.index() % width) * 50 
    GA.y[n] = (n.index() // height) * 50

box = GA.boundingBox()
GA, wraps = layout_edges(GA, box)
# draw_bounding_box(G, GA, box)
GA

In [4]:
for e in G.edges:
    print(e.source().index(),e.target().index(),find_wrap(GA,e))
    

0 1 
1 2 
2 3 
3 0 h
0 4 
4 5 
1 5 
5 6 
2 6 
6 7 
3 7 
7 4 h
4 8 
8 9 
5 9 
9 10 
6 10 
10 11 
7 11 
11 8 h
8 12 
12 13 
9 13 
13 14 
10 14 
14 15 
11 15 
15 12 h
12 0 v
13 1 v
14 2 v
15 3 v


In [5]:
G.delNode(G.nodes[0])
G.delNode(G.nodes[1])
G.delNode(G.nodes[2])
G.delNode(G.nodes[4])
G.delNode(G.nodes[8])
G.delNode(G.nodes[12])
G.delNode(G.nodes[13])
G.delNode(G.nodes[11])
G.delNode(G.nodes[14])
G.delNode(G.nodes[15])


G.delEdge(G.searchEdge(G.nodes[5], G.nodes[6]))

# G.unsplit(G.nodes[15]) # replace deg 2 node by edge

G.newEdge(G.nodes[9],G.nodes[5])
G.newEdge(G.nodes[3],G.nodes[5])
G.newEdge(G.nodes[10],G.nodes[6])
G.newEdge(G.nodes[7],G.nodes[3])




layout_edges(GA, box)
GA

In [6]:
def adjList(nodes):
    
    adjList = {}
    
    for n in G.nodes:
        
        #adjList[GA.label[n]] = {}
        adjList[n] = {}
        
        for adj in n.adjEntries:
            
            e = adj.theEdge()
            wrap = find_wrap(GA, e)
            
            if n == e.source():
                
                if wrap == "h" or wrap == "hv":
                    position = 1
                    
                elif wrap == "v" or wrap == "vh":
                    position = 2
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 2
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 1
        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node lab
                adjList[n][position] = adj
                        
            else:
                
                if wrap == "h" or wrap == "hv":
                    position = 3
                    
                elif wrap == "v" or wrap == "vh":
                    position = 0
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 0
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 3
                        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node labels
                adjList[n][position] = adj
                
    
    return adjList

result = adjList(G.nodes)

result

{<cppyy.gbl.ogdf.NodeElement object at 0x5f43da0>: {2: <cppyy.gbl.ogdf.AdjElement object at 0x5f2a9f0>,
  1: <cppyy.gbl.ogdf.AdjElement object at 0x5f2b200>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x5f2af30>},
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43e20>: {2: <cppyy.gbl.ogdf.AdjElement object at 0x5f2ab70>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x5f2a8d0>,
  3: <cppyy.gbl.ogdf.AdjElement object at 0x5f2b1d0>},
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43e60>: {1: <cppyy.gbl.ogdf.AdjElement object at 0x5f2a990>,
  2: <cppyy.gbl.ogdf.AdjElement object at 0x5f2ac30>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x5f2af90>},
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43ea0>: {3: <cppyy.gbl.ogdf.AdjElement object at 0x5f2a9c0>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x5f2aa20>,
  2: <cppyy.gbl.ogdf.AdjElement object at 0x5f2af60>},
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43f20>: {0: <cppyy.gbl.ogdf.AdjElement object at 0x5f2aba0>,
  1: <cppyy.gbl.ogdf.AdjElement object at 0x

In [7]:
def sorted_adjList(adjList):
    
    sorted_adjList = {}
    
    for n in adjList:
        
        sortedList = []
        
        for i in sorted (adjList[n]) :
            
            sortedList.append(adjList[n][i])
        
        sorted_adjList[n] = sortedList
    
    return sorted_adjList
    
    
sorted_adjList = sorted_adjList(result)
sorted_adjList

{<cppyy.gbl.ogdf.NodeElement object at 0x5f43da0>: [<cppyy.gbl.ogdf.AdjElement object at 0x5f2af30>,
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43e20>: [<cppyy.gbl.ogdf.AdjElement object at 0x5f2a8d0>,
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43e60>: [<cppyy.gbl.ogdf.AdjElement object at 0x5f2af90>,
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43ea0>: [<cppyy.gbl.ogdf.AdjElement object at 0x5f2aa20>,
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43f20>: [<cppyy.gbl.ogdf.AdjElement object at 0x5f2aba0>,
 <cppyy.gbl.ogdf.NodeElement object at 0x5f43f60>: [<cppyy.gbl.ogdf.AdjElement object at 0x5f2ac60>,
  <cppyy.gbl.ogdf.AdjElement object at 0x5f2ac00>]}

In [8]:
def sortAdjs(node, order):
    l = ogdf.List["ogdf::adjEntry"]()
    for adj in order:
        l.pushBack(adj)
    node.adjEntries.sort(l)

node3 = list(sorted_adjList)[0]
node3_adjEntries = sorted_adjList[node3]
sortAdjs(node3, node3_adjEntries)
list(node3.adjEntries)


In [9]:
for n in sorted_adjList:
    
    node_adjEntries = sorted_adjList[n]
    sortAdjs(n, node_adjEntries)
    print(list(n.adjEntries))

   

[<cppyy.gbl.ogdf.AdjElement object at 0x5f2af30>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2b200>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2a9f0>]
[<cppyy.gbl.ogdf.AdjElement object at 0x5f2a8d0>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2ab70>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2b1d0>]
[<cppyy.gbl.ogdf.AdjElement object at 0x5f2af90>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2a990>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2ac30>]
[<cppyy.gbl.ogdf.AdjElement object at 0x5f2aa20>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2af60>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2a9c0>]
[<cppyy.gbl.ogdf.AdjElement object at 0x5f2aba0>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2abd0>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2a900>]
[<cppyy.gbl.ogdf.AdjElement object at 0x5f2ac60>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2afc0>, <cppyy.gbl.ogdf.AdjElement object at 0x5f2ac00>]


In [10]:
G.genus()

1

In [11]:
#for computing the dual

import cppyy
dualG = ogdf.Graph()
# maps G.adjEntry -> dualG.node (face)
dualFace = ogdf.AdjEntryArray["ogdf::node"](G, cppyy.bind_object(cppyy.nullptr, "ogdf::NodeElement"))
# maps G.edge -> dualG.edge
dualEdge = ogdf.EdgeArray["ogdf::edge"](G, cppyy.bind_object(cppyy.nullptr, "ogdf::EdgeElement"))
# maps dualG.edge -> G.edge (== dualEdge^{-1})
primalEdge = ogdf.EdgeArray["ogdf::edge"](dualG, cppyy.bind_object(cppyy.nullptr, "ogdf::EdgeElement"))

for n in G.nodes:
    for adj in n.adjEntries:
        if dualFace[adj]:
            continue
        face = dualG.newNode()
        dualFace[adj] = face
        
        a = adj.faceCycleSucc()
        while a != adj:
            dualFace[a] = face
            a = a.faceCycleSucc()

for e in G.edges:
    de = dualG.newEdge(dualFace[e.adjSource()], dualFace[e.adjTarget()])
    dualEdge[e] = de
    primalEdge[de] = e

In [12]:
#for computing the spanning trees 

for e in G.edges:
    GA.strokeColor[e] = ogdf.Color(0,0,0)

# type of G edges: 0 (black) is in X, 1 (red) is edge of spanning cotree, 2 (green) is edge of primal ST
tree = ogdf.EdgeArray[int](G, 0)

found = ogdf.NodeArray[bool](dualG, False)
first_node = next(iter(G.nodes))
found[first_node] = True
todo = list(dualG.nodes[0].adjEntries)
count = 1
while len(todo) > 0:
    cur = todo.pop(-1)
    if found[cur.twinNode()]:
        continue

    tree[primalEdge[cur.theEdge()]] = 1
    GA.strokeColor[primalEdge[cur.theEdge()]] = ogdf.Color(255,0,0)
    found[cur.twinNode()] = True
    count += 1

    for adj in cur.twinNode().adjEntries:
        if not found[adj.twinNode()]:
            todo.append(adj)
print(count, dualG.numberOfNodes())

found = ogdf.NodeArray[bool](G, False)
first_node = next(iter(G.nodes))
found[first_node] = True
todo = list(first_node.adjEntries)
count = 1
while len(todo) > 0:
    cur = todo.pop(-1)
    if found[cur.twinNode()] or tree[cur] != 0:
        continue

    tree[cur.theEdge()] = 2
    GA.strokeColor[cur.theEdge()] = ogdf.Color(0,255,0)
    found[cur.twinNode()] = True
    count += 1

    for adj in cur.twinNode().adjEntries:
        if not found[adj.twinNode()] and tree[adj] == 0:
            todo.append(adj)

print(count, G.numberOfNodes())

unlabeled = [e for e in G.edges if tree[e] == 0]
print(unlabeled)
GA


4 3
6 6
[<cppyy.gbl.ogdf.EdgeElement object at 0x5f3d148>, <cppyy.gbl.ogdf.EdgeElement object at 0x5f3cff8>]


In [13]:
for e in G.edges:
    if tree[e] == 0:
        start_edge = e
        break
    

In [14]:
#start from any edge, and any of its node, check adjacent edges of the node, if already set, go to the next

import math
orientation = ogdf.EdgeArray[int](G,99)

def bendCoord(e):
    bends = [(b.m_x,b.m_y) for b in GA.bends[e]]
    return bends

def findAngle(node,x,y):
    refx = GA.x[node]
    refy = GA.y[node]
    rad = math.atan2(y-refy,x-refx)
    deg = math.degrees(rad)
    return deg

def defineOrientation(refEdge,refNode,refOrientation):
    #print("new refEdge: ", "source: ", refEdge.source().index(), "target: ", refEdge.target().index())
    
    if len(bendCoord(refEdge)) != 0:
        if refEdge.target() == refNode:          
            bendpoint = bendCoord(refEdge)[-1]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            refAngle = findAngle(refNode,bendpointx,bendpointy)
                        
        else:
            bendpoint = bendCoord(refEdge)[0]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            refAngle = findAngle(refNode,bendpointx,bendpointy)
                        
    else:
        if refEdge.target() == refNode:        
            refAngle = findAngle(refNode,GA.x[refEdge.source()],GA.y[refEdge.source()])
                        
        else:         
            refAngle = findAngle(refNode,GA.x[refEdge.target()],GA.y[refEdge.target()])
        
    
    
    for adj in refNode.adjEntries:
        e= adj.theEdge()
        nextNode = adj.twinNode()
        #print(refNode.index(),nextNode.index())
        if e!= refEdge:
            if orientation[e] == 0 or orientation [e] == 1:
                #print("this edge is done")
                continue
                
            else:
                if len(bendCoord(e)) != 0:
                    #print("has bend")
                    if e.target() == refNode:
                    
                        bendpoint = bendCoord(e)[-1]
                        bendpointx = bendpoint[0]
                        bendpointy = bendpoint[1]
                    
                        angle = findAngle(refNode,bendpointx,bendpointy)
                        
                    else:
                        bendpoint = bendCoord(e)[0]
                        bendpointx = bendpoint[0]
                        bendpointy = bendpoint[1]
                    
                        angle = findAngle(refNode,bendpointx,bendpointy)
                        
                else:
                    angle = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
                    
                    
                
            if abs(angle)%180 == abs(refAngle)%180:
                orientation[e] = refOrientation
                    
            else: 
                orientation[e] = 1-refOrientation
                    
            #print("orientation of this edge is ", orientation[e], " angle of this edge is ", angle)
            
                    
            defineOrientation(e,nextNode,orientation[e])
    
    return
   
        
        
def findGenerators(start_edge, source, target, generator):
   
    for adj in target.adjEntries:
        
        if tree[adj.theEdge()] == 2 and adj.theEdge() != start_edge:
            cur_edge = adj.theEdge()
            next_node = adj.twinNode()
            generator.append(cur_edge)
        
            if next_node == source:
                return True, generator
            else:
                bool, generator = findGenerators(cur_edge, source, next_node,generator)
                if bool:
                    return True, generator
        
    generator.remove(start_edge)
    return False, generator
        

In [15]:
defineOrientation(start_edge,start_edge.source(),0)
for e in G.edges:
    print(e.source().index(), e.target().index(), orientation[e])

6 7 0
3 7 1
5 9 1
9 10 0
6 10 1
9 5 1
3 5 0
10 6 1
7 3 1


In [16]:
#the black edges that must be included in each generator
starter_edges = [e for e in G.edges if tree[e] == 0]


start_edge = starter_edges[0]
betaLoop = [start_edge]
bool, betaLoop = findGenerators(start_edge, start_edge.source(),start_edge.target(), betaLoop)
print("beta loop")
for i, e in enumerate(betaLoop):
    print(betaLoop[i].source().index(),betaLoop[i].target().index())

print("alpha loop")
start_edge = starter_edges[1]
alphaLoop = [start_edge]
bool, alphaLoop = findGenerators(start_edge, start_edge.source(),start_edge.target(),alphaLoop)
for i, e in enumerate(alphaLoop):
    print(alphaLoop[i].source().index(),alphaLoop[i].target().index())

beta loop
3 5
5 9
9 10
6 10
6 7
3 7
alpha loop
10 6
6 10


def findGenerators(start_edge, source, target):
    #print(source.index(),target.index())
    for adj in target.adjEntries:
        
        if tree[adj.theEdge()] == 2 and adj.theEdge() != start_edge:
            cur_edge = adj.theEdge()
            next_node = adj.twinNode()
            #print(next_node.index())
            generator.append(cur_edge)
        
            if next_node == source:
                return True
            else:
                if findGenerators(cur_edge, source, next_node):
                    return True
        
    generator.remove(start_edge)
    return False

generator = [starter_edges[0]]
findGenerators(starter_edges[0], starter_edges[0].source(),starter_edges[0].target())
for i, e in enumerate(generator):
    print(generator[i].source().index(),generator[i].target().index())

from pulp import *
import pandas as pd
import numpy as np

model = LpProblem('Network Flow', LpMinimize)

alphaLoopHorizontal = []
alphaLoopVertical = []
betaLoopHorizontal = []
betaLoopVertical = []

for e in alphaLoop:
    if orientation[e] == h:
        alphaLoopHorizontal.append(e)
    else:
        alphaLoopVertical.append(e)

for e in betaLoop:
    if orientation[e] == h:
        betaLoopHorizontal.append(e)
    else:
        betaLoopVertical.append(e)

alphaHorizontalDecisionVariables = []
alphaVerticalDecisionVariables = []
betaHorizontalDecisionVariable = []
betaVerticalDecisionVariables = []
for i, e in enumerate(alphaLoopHorizontal):
    var = LpVariable('eah' + str(i) ,lowBound = 0) #change
    alphaHorizontalDecisionVariables.append(var)
    
    if e in betaLoopHorizontal:
        betaHorizontalDecisionVariables.append(var)
        
for i, e in enumerate(alphaLoopVertical):
    var = LpVariable('eav' + str(i) ,lowBound = 0) #change
    alphaVerticalDecisionVariables.append(var)
    
    if e in betaLoopVertical:
        betaVerticalDecisionVariables.append(var)
        
for i, e in enumerate(betaLoopHorizontal):
    var = LpVariable('ebh' + str(i) ,lowBound = 0) #change
   
    if e not in alphaLoopHorizontal:
        betaHorizontalDecisionVariables.append(var) 
        
for i, e in enumerate(betaLoopVertical):
    var = LpVariable('ebv' + str(i) ,lowBound = 0) #change
   
    if e not in alphaLoopVertical:
        betaVerticalDecisionVariables.append(var) 
        
        
        
sumVariables = alphaHorizontalDecisionVariables + betaHorizontalDecisionVariable
                + alphaVerticalDecisionVariables + betaVerticalDecisionVariables
sumCoeff = np.full(
  shape=len(sumVariables),
  fill_value=1,
  dtype=np.int
)

alphaHorzCoeff = np.full(
  shape=len(alphaHorizontalDecisionVariables),
  fill_value=1,
  dtype=np.int
)
betaHorzCoeff = np.full(
  shape=len(betaHorizontalDecisionVariables),
  fill_value=1,
  dtype=np.int
)
alphaVerCoeff = np.full(
  shape=len(alphaVerticalDecisionVariables),
  fill_value=1,
  dtype=np.int
)
betaVerCoeff = np.full(
  shape=len(betaVerticalDecisionVariables),
  fill_value=1,
  dtype=np.int
)
obj_func = lpSum(sumVariables*sumCoeff)
model +=  obj_func

model += lpSum(alphaHorizontalDecisionVariables*alphaHorzCoeff) == 1 , "Constraint 1" 
model += lpSum(betaHorzDecisionVariables*betaHorzCoeff) == 1 , "Constraint 2"
model += lpSum(alphaVerDecisionVariables*alphaVerCoeff) == 1 , "Constraint 3" 
model += lpSum(betaVerDecisionVariables*betaVerCoeff) == 1 , "Constraint 4"

model.writeLP('Network Flow.lp')

model.solve(PULP_CBC_CMD())

status =  LpStatus[model.status]

print(status)

In [17]:
signs = {}
signs[alphaLoop[0]] = 1
i=0
flip = False
while i < len(alphaLoop)-1:
    if flip == False:
        s1 = alphaLoop[i].source()
        t1 = alphaLoop[i].target()
        s2 = alphaLoop[i+1].source()
        t2 = alphaLoop[i+1].target()
    else:
        s1 = alphaLoop[i].target()
        t1 = alphaLoop[i].source()
        s2 = alphaLoop[i+1].source()
        t2 = alphaLoop[i+1].target()
    
    if t1 == s2: #if current target of edge is the source of next edge
        signs[alphaLoop[i+1]] = 1
        flip = False
        
    else:
        signs[alphaLoop[i+1]] = -1
        flip = True
    
        
    i += 1
    

signs[betaLoop[0]] = 1
i=0   
flip = False
while i < len(betaLoop)-1:
    #print(betaLoop[i].source().index(),betaLoop[i].target().index())
    if flip == False:
        s1 = betaLoop[i].source()
        t1 = betaLoop[i].target()
        s2 = betaLoop[i+1].source()
        t2 = betaLoop[i+1].target()
    else:
        s1 = betaLoop[i].target()
        t1 = betaLoop[i].source()
        s2 = betaLoop[i+1].source()
        t2 = betaLoop[i+1].target()
    
    if t1 == s2: #if current target of edge is the source of next edge
        signs[betaLoop[i+1]] = 1
        flip = False
        
    else:
        signs[betaLoop[i+1]] = -1
        flip = True
    
        
    i += 1


for i, e in enumerate(alphaLoop):
    print(signs[e],alphaLoop[i].source().index(),alphaLoop[i].target().index())
    
for i, e in enumerate(betaLoop):
    print(signs[e],betaLoop[i].source().index(),betaLoop[i].target().index())

1 10 6
-1 6 10
1 3 5
1 5 9
1 9 10
-1 6 10
1 6 7
-1 3 7


In [18]:
from pulp import *
import pandas as pd
import numpy as np
alphaLoopVertical = []
for e in alphaLoop:
    if orientation[e] == 'h':
        alphaLoopHorizontal.append(e)
    else:
        alphaLoopVertical.append(e)
        
alphaHorizontalDecisionVariables = {}
alphaVerticalDecisionVariables = {}
betaHorizontalDecisionVariable = {}
betaVerticalDecisionVariables = {}
for i, e in enumerate(alphaLoopVertical):
    var = LpVariable('eav' + str(i) ,lowBound = 0) #change
    alphaVerticalDecisionVariables[e] = var
    
    #if e in betaLoopVertical:
     #   betaVerticalDecisionVariables.append(var)
        
model = LpProblem('Network Flow', LpMinimize)
x1 = LpVariable("x1", 0)
x2 = LpVariable("x2", 0)
model +=  signs[0]*x1 + signs[1]*x2, "Sum of flow"
model += signs[0] *x1 + signs[1]*x2 == 1, "Constraint 1"
model += lpSum([signsAlpha[e] * x[table] for table in possible_tables])
model.solve()
print("Status:", LpStatus[model.status])

/home/loke/.local/lib/python3.9/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


KeyError: 0

Example LP problem

In [ ]:
prob = LpProblem("The Whiskas Problem", LpMinimize)

# The 2 variables Beef and Chicken are created with a lower limit of zero
x1 = LpVariable("ChickenPercent", 0, None, LpInteger)
x2 = LpVariable("BeefPercent", 0)

# The objective function is added to 'prob' first
prob += 0.013 * x1 + 0.008 * x2, "Total Cost of Ingredients per can"

# The five constraints are entered
prob += x1 + x2 == 100, "PercentagesSum"
prob += 0.100 * x1 + 0.200 * x2 >= 8.0, "ProteinRequirement"
prob += 0.080 * x1 + 0.100 * x2 >= 6.0, "FatRequirement"
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, "FibreRequirement"
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, "SaltRequirement"

prob.solve()
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = ", value(prob.objective))